<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU25_Modelo de gestión de Lista de Espera Quirúrgica</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>09.2.- Data Cleansing - Missing</font>

<font color='steelblue'>Data Cleaning refers to identifying and correcting (or removing) errors in the dataset that may negatively impact a predictive model, replacing, modifying, or deleting the dirty or coarse data.</font>

## <font color='steelblue'>Tasks</font>

## Consideraciones casos CitizenLab programados en R

* La mayoría de las tareas de este proceso se han realizado en los notebooks del proceso 05 Data Collection porque eran necesarias para las tareas ETL. En esos casos, en este notebook se referencia al notebook del proceso 05 correspondiente
* Por tanto en los notebooks de este proceso de manera general se incluyen las comprobaciones necesarias, y comentarios si procede
* Las tareas del proceso se van a aplicar solo a los archivos que forman parte del despliegue, ya que hay muchos archivos intermedios que no procede pasar por este proceso
* El nombre de archivo del notebook hace referencia al nombre de archivo del proceso 05 al que se aplica este proceso, por eso pueden no ser correlativa la numeración
* Las comprobaciones se van a realizar teniendo en cuenta que el lenguaje utilizado en el despliegue de este caso es R

## <font color='green'>File</font>

- <font color='steelblue'> <b>Input File: CU_25_09.1_01_lista_espera_completo_clean_v_01.csv</font>
- <font color='steelblue'> <b>Output File: No aplica</font>

## <font color='green'>Settings</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [27]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "LC_COLLATE=es_ES.UTF-8;LC_CTYPE=es_ES.UTF-8;LC_MONETARY=es_ES.UTF-8;LC_NUMERIC=C;LC_TIME=es_ES.UTF-8"

### Libraries to use

In [28]:
library(readr)
library(dplyr)
library(sf)
library(tidyr)
library(stringr)
library(zoo)
library(imputeTS)

### Paths

In [29]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if using this option

In [30]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [31]:
iFile <- "CU_25_09.1_01_lista_espera_completo_clean_v_01.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_25_09.1_01_lista_espera_completo_clean_v_01.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [32]:
data <- read.csv(file_data)


<font color='tomato'>Visualizo los datos.</font>

Estructura de  los datos:

In [33]:
data |> glimpse()

Rows: 55,680
Columns: 46
$ Hospital          <chr> "HOSPITAL REY JUAN CARLOS", "HOSPITAL CENTRAL DE LA …
$ Especialidad      <chr> "UROLOGÍA", "ODONTOESTOMATOLOGÍA", "GINECOLOGÍA", "D…
$ total_pacientes   <int> 344, 0, 52, 37, 0, 4, 0, 718, 0, 271, 108, 0, 34, 86…
$ ano               <int> 2021, 2020, 2021, 2021, 2021, 2020, 2021, 2020, 2021…
$ semana            <int> 30, 36, 49, 23, 3, 5, 50, 7, 35, 1, 42, 10, 21, 33, …
$ CODCNH            <int> 281348, 280724, 281292, 281292, 281236, 280724, 2807…
$ id_area           <int> 8, 7, 11, 11, 11, 7, 3, 6, 1, 2, 2, 8, 11, 11, 1, 3,…
$ nombre_area       <chr> "SUR-OESTE I", "CENTRO-OESTE", "SUR II", "SUR II", "…
$ cmunicipio        <int> 280920, 280796, 280133, 280133, 281610, 280796, 2800…
$ Municipio         <chr> "MÓSTOLES", "MADRID", "ARANJUEZ", "ARANJUEZ", "VALDE…
$ CAMAS             <int> 382, 475, 98, 98, 182, 475, 507, 613, 269, 1143, 156…
$ Clase             <chr> "HOSPITALES GENERALES", "HOSPITALES GENERALES", "HOS…
$ Dependencia  

Muestra de los primeros datos:

In [34]:
data |> slice_head(n = 5)

Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t9_1,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
HOSPITAL REY JUAN CARLOS,UROLOGÍA,344,2021,30,281348,8,SUR-OESTE I,280920,MÓSTOLES,⋯,0.3348780,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE
HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,ODONTOESTOMATOLOGÍA,0,2020,36,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.4914365,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,GINECOLOGÍA,52,2021,49,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,DERMATOLOGÍA,37,2021,23,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE
HOSPITAL UNIVERSITARIO INFANTA ELENA,ODONTOESTOMATOLOGÍA,0,2021,3,281236,11,SUR II,281610,VALDEMORO,⋯,0.3346063,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE


## Missing Values

### Missing Values Identification

#### Missing Values Per Sample

In [35]:
# Create a dataframe with ID and number of missing values per sample
missing_per_sample_df <- data.frame(ID = 1:nrow(data), NAs = rowSums(is.na(data)))

# Filter rows with more than one missing value
filtered_missing_per_sample_df <- missing_per_sample_df[missing_per_sample_df$NAs > 1, ]

# Print the filtered dataframe
print(filtered_missing_per_sample_df)

         ID NAs
138     138   4
241     241   4
577     577   4
579     579   4
614     614   4
759     759   4
786     786   4
951     951   4
1087   1087   4
1183   1183   4
1348   1348   4
1487   1487   4
1920   1920   4
2049   2049   4
2081   2081   4
2102   2102   4
2351   2351   4
2414   2414   4
2535   2535   4
2981   2981   4
3115   3115   4
3362   3362   4
3367   3367   4
3445   3445   4
3475   3475   4
3495   3495   4
3542   3542   4
3587   3587   4
3672   3672   4
3745   3745   4
3905   3905   4
4030   4030   4
4166   4166   4
4287   4287   4
4352   4352   4
4611   4611   4
4660   4660   4
4799   4799   4
5021   5021   4
5023   5023   4
5051   5051   4
5073   5073   4
5193   5193   4
5196   5196   4
5199   5199   4
5401   5401   4
5484   5484   4
5709   5709   4
5803   5803   4
6016   6016   4
6092   6092   4
6144   6144   4
6292   6292   4
6391   6391   4
6491   6491   4
6556   6556   4
6661   6661   4
6680   6680   4
6690   6690   4
6757   6757   4
7052   7052   4
7055   7

#### Missing Values Per Feature

In [36]:
# Calculate the number of missing values per feature
missing_per_feature <- colSums(is.na(data))

# Print the number of missing values per feature
print(missing_per_feature)

         Hospital      Especialidad   total_pacientes               ano 
                0                 0               464                 0 
           semana            CODCNH           id_area       nombre_area 
                0                 0                 0                 0 
       cmunicipio         Municipio             CAMAS             Clase 
                0                 0                 0                 0 
      Dependencia               TAC                RM               GAM 
                0                 0                 0                 0 
              HEM               ASD               ALI             SPECT 
                0                 0                 0                 0 
            MAMOS                DO              DIAL                 X 
                0                 0                 0                 0 
                Y              t3_1              t1_1              t2_1 
                0                 0                

#### Zero Missing Values

In [37]:
# Detecting columns with minimum value of zero (0).

# Calculate the frequency of zeros in each column
zero_counts <- sapply(data, function(x) sum(x == 0, na.rm = TRUE))

# Calculate the proportion of zeros in each column
zero_proportions <- zero_counts / nrow(data)

# Set a threshold for the proportion of zeros
zero_threshold <- 0.9  # Adjust as needed

# Identify columns with a high proportion of zeros
columns_with_high_zeros <- names(zero_proportions[zero_proportions >= zero_threshold])

# Print the columns with a high proportion of zeros
print(columns_with_high_zeros)

character(0)


<font color='tomato'>Select column to replace</font>

In [38]:
# Select column to replace
#BCO

<font color='darkgrey'><b>Operation</font>

In [39]:
# Replace zero missing values by nan
# Replace columns with a high proportion of zeros with NaN
data[, columns_with_high_zeros] <- NA

#### Other Missing Values

<font color='tomato'>Select column to replace</font>

In [40]:
# Select column to replace and missing value


<font color='darkgrey'><b>Operation</font>

In [41]:
# Replace other missing values by nan


#### Null/NaN Missing Values

In [42]:
# Intuitivamente: miramos nº datos en todas las columnas
# los null no los cuenta --> debe hacer el mismo nº por columna


In [43]:
# Podemos mirar directamente info donde viene


In [44]:
# Contamos los nulos de forma explícita


In [45]:
# summarize the number of rows with missing values for each column


### Delete Missing Values

#### Deleting Rows with Missing Values in Target Column

In [46]:
data <- data[!is.na(data$Target), ]

#### Deleting Rows with Missing Values
Only in case of high data size

In [47]:
# Eliminamos las filas con valores nulos


#### Deleting Features with some Missing Values
Only with many features and for non-relevant features

In [48]:
# Selecciono las columnas con algún valor missing:
# Delete features with missing values
clean_data <- data %>%
  select(where(~ !any(is.na(.))))

#### Deleting Features using Rate Missing Values

In [49]:
# Number of data
# Define the minimum number of non-missing values
n <- 10  # For example, at least 10 non-missing values

# Count the number of non-missing values for each feature
non_missing_counts <- colSums(!is.na(data))

# Get the names of features that have fewer than n non-missing values
features_to_delete <- names(non_missing_counts[non_missing_counts < n])

# Remove the features with fewer than n non-missing values
data <- data[, !(names(data) %in% features_to_delete)]

In [50]:
# Number of missing data
n<- 10  # For example, at least 10 missing values
# Count the number of missing values for each feature
missing_counts <- colSums(is.na(data))

# Get the names of features that have missing values
features_to_delete <- names(missing_counts[missing_counts > n])

# Remove the features with missing values
data <- data[, !(names(data) %in% features_to_delete)]

In [51]:
# Rate (%) of missing data 
# Define the rate threshold for missing values
rate_threshold <- 0.6  # For example, 60% missing values

# Calculate the percentage of missing values for each feature
missing_rate <- colMeans(is.na(data))

# Get the names of features that exceed the rate threshold
features_to_delete <- names(missing_rate[missing_rate > rate_threshold])

# Remove the features with high missing value rates
data <- data[, !(names(data) %in% features_to_delete)]

### Basic Imputation

#### Imputation by Previous Row Value

In [52]:
imputed_data

ERROR: Error in eval(expr, envir, enclos): object 'imputed_data' not found


In [ ]:
# Sustituimos valores null por otro valor: VALOR FILA ANTERIOR

# Replace null values with the previous row's value
imputed_data <- data %>%
  mutate(across(.cols = everything(), .fns = ~ if_else(is.na(.), lag(.), .)))

#### Imputation by Next Row Value

In [ ]:
# Sustituimos valores null por otro valor: VALOR FILA SIGUIENTE
imputed_data <- data
for (col in names(imputed_data)) {
  missing_indices <- which(is.na(imputed_data[[col]]))
  for (i in missing_indices) {
    next_value <- imputed_data[[col]][i+1]
    imputed_data[[col]][i] <- next_value
  }
}


### Statistical Imputation

A popular approach for data imputation is to calculate a statistical value for each column (such as a mean) and replace all missing values for that column with the 
statistic.

#### Selection of Imputation Strategy

In [ ]:
# The mean accuracy of each approach can then be compared.
#
# Specific results may vary given the stochastic nature of 
# the learning algorithm, the evaluation procedure, or 
# differences in numerical precision. Consider running the 
# example a few times and compare the average performance.
#


In [ ]:
# Plot model performance for comparison
# box and whisker plot is created for each set of results, 
# allowing the distribution of results to be compared.


#### Constant Imputation

<font color='tomato'>Select constant value</font>

In [ ]:
# Constant imputation
# Replace missing values with constant values
str_impute<- "impute"
numeric_impute <- 0
imputed_data <- data %>%
  mutate_if(is.character, ~ifelse(is.na(.), str_impute, .)) %>%
  mutate_if(is.numeric, ~ifelse(is.na(.), numeric_impute, .))

#### Mean Imputation

In [ ]:
# Sustituimos valores null por otro valor: MEDIA


In [ ]:
# Sustituyo
# Impute missing values with the column means

imputed_data <- data %>%
  mutate(across(where(is.numeric), ~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))

#### Median Imputation

In [ ]:
# Sustituimos valores null por otro valor: MEDIA
# Miro la mediana


In [ ]:
# Sustituyo

imputed_data <- data %>%
  mutate(across(where(is.numeric), ~ ifelse(is.na(.), median(., na.rm = TRUE), .)))

#### Most Frequent Imputation

In [ ]:
# Sustituyo
get_mode <- function(x) {
  freq_table <- table(x)
  mode <- as.numeric(names(freq_table)[freq_table == max(freq_table)])
  return(mode)
}

imputed_data <- data %>%
  mutate(across(everything(), ~ ifelse(is.na(.),get_mode(.), .)))



#### Interpolation Imputation

In [ ]:
# Sustituimos valores null por otro valor: INTERPOLANDO
# Métodos de interpolación
# ‘linear’, ‘time’, ‘index’, ‘values’, ‘nearest’, ‘zero’, ‘slinear’, 
# ‘quadratic’, ‘cubic’, ‘barycentric’, ‘krogh’, ‘polynomial’, ‘spline’ 
# ‘piecewise_polynomial’, ‘pchip’


### Prediction Imputation (KNN Imputation )
An approach to missing data imputation is to use a model to predict the missing values.

#### Evaluating k-hyperparmeter in KNN Imputation

<font color='tomato'>Select numbers of neighbors to evaluate</font>

In [ ]:
# Numbers of neighbors to evaluate
k = 5


<font color='darkgrey'><b>Operation</font>

#### Applying KNN Imputation 

In [ ]:
imputed_data <- data %>%
  mutate(across(where(is.numeric), ~ ifelse(is.na(.), impute_knn(., k = k), .)))

<font color='tomato'>Select numbers of neighbors to evaluate</font>

In [ ]:
# Generating de new Data dataframe


### Iterative Imputation 

#### Evaluating Different Imputation Order
We can experiment with different imputation order strategies, such as descending, right-to-left (Arabic), left-to-right (Roman), and random.

In [ ]:
# compare iterative imputation strategies for the horse colic dataset


#### Applying Iterative Imputation

<font color='tomato'>Select strategie</font>

In [ ]:
# Selecting strategie
# strategies = ['ascending', 'descending', 'roman', 'arabic', 'random']


ERROR: Error in impute_knn(): could not find function "impute_knn"


<font color='darkgrey'><b>Operation</font>

In [ ]:
# Generating the new Data dataframe


## <font color='green'>Data Save</font>

* Solo si se han hecho cambios

* No aplica

<font color='tomato'> Identificamos los datos a guardar</font>

In [53]:
data_to_save <- data

         Hospital      Especialidad   total_pacientes               ano 
                0                 0                 0                 0 
           semana            CODCNH           id_area       nombre_area 
                0                 0                 0                 0 
       cmunicipio         Municipio             CAMAS             Clase 
                0                 0                 0                 0 
      Dependencia               TAC                RM               GAM 
                0                 0                 0                 0 
              HEM               ASD               ALI             SPECT 
                0                 0                 0                 0 
            MAMOS                DO              DIAL                 X 
                0                 0                 0                 0 
                Y              t3_1              t1_1              t2_1 
                0                 0                


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_06".
* Resto del nombre del archivo de entrada
* Extensión del archivo

Ejemplo: "CU_04_06_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas) y que se ha transformado en el proceso 06

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 09.2

In [54]:
caso <- "CU_25"
proceso <- '_09.2'
tarea <- "_01"
archivo <- "_lista_espera_completo_clean"
proper <- "_v_01"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [ ]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [55]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_25_09.2_01_lista_espera_completo_clean_v_01.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [56]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color='#2874a6'>REPORT</font>

A continuación se realizará un informe de las acciones realizadas

## <font color=' #2874a6 '>Main Actions Carried Out</font>

- Si eran necesarias se han realizado en el proceso 05 por cuestiones de eficiencia

## <font color=' #2874a6 '>Main Conclusions</font>

- Los datos están limpios para el despliegue


## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>